In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = 'output_data/cities.csv'

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        
#OpenWeather Key
api_key = weather_api
units = "imperial"

query_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key}&units={units}&q="

# Print the city count to confirm sufficient count
len(cities)

620

In [3]:
#Empty lists for my variables
cloudiness = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []
country = []
city_column = []

count = 0
sets = 1

In [4]:
#cities


In [5]:
#Run the API call for each city
print('Beginning Data Retrieval')
print('-----------------------------')

for city in cities:
    weather_json = requests.get(query_url + city.replace(" ","&")).json()
    if count < 50:
            count += 1
            city_column.append(city)
    else:
            sets += 1
            count = 0
            city_column.append(city)
    print(f"Processing Record {count} of Set {sets} | {city}")
    try:
        cloudiness.append(weather_json['clouds']['all'])
        country.append(weather_json['sys']['country'])
        date.append(weather_json['dt'])
        humidity.append(weather_json['main']['humidity'])
        max_temp.append(weather_json['main']['temp_max'])
        lat.append(weather_json['coord']['lat'])
        lng.append(weather_json['coord']['lon'])
        wind_speed.append(weather_json['wind']['speed'])
        city_column.append(city)
    except:
        print('City not found. Skipping...')
    pass
print('------------------------------')
print('Data Retrieval Complete')
print('------------------------------')

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | yumen
Processing Record 3 of Set 1 | halifax
Processing Record 4 of Set 1 | luwuk
Processing Record 5 of Set 1 | hay river
Processing Record 6 of Set 1 | arraial do cabo
Processing Record 7 of Set 1 | kaitangata
Processing Record 8 of Set 1 | esperance
Processing Record 9 of Set 1 | los llanos de aridane
City not found. Skipping...
Processing Record 10 of Set 1 | pacific grove
Processing Record 11 of Set 1 | saskylakh
Processing Record 12 of Set 1 | kapaa
Processing Record 13 of Set 1 | grand river south east
Processing Record 14 of Set 1 | caranavi
Processing Record 15 of Set 1 | ambilobe
Processing Record 16 of Set 1 | geraldton
Processing Record 17 of Set 1 | artyom
Processing Record 18 of Set 1 | highland springs
Processing Record 19 of Set 1 | ushuaia
Processing Record 20 of Set 1 | jamestown
Processing Record 21 of Set 1 | yellowknife
Processing Record 22

Processing Record 34 of Set 4 | umzimvubu
City not found. Skipping...
Processing Record 35 of Set 4 | avarua
Processing Record 36 of Set 4 | tasiilaq
Processing Record 37 of Set 4 | aitape
Processing Record 38 of Set 4 | bridlington
Processing Record 39 of Set 4 | kitimat
Processing Record 40 of Set 4 | pangai
Processing Record 41 of Set 4 | nouakchott
Processing Record 42 of Set 4 | grindavik
Processing Record 43 of Set 4 | homer
Processing Record 44 of Set 4 | hithadhoo
Processing Record 45 of Set 4 | severnyy-kospashskiy
Processing Record 46 of Set 4 | salalah
Processing Record 47 of Set 4 | yeppoon
Processing Record 48 of Set 4 | praia da vitoria
Processing Record 49 of Set 4 | karasjok
Processing Record 50 of Set 4 | lasa
Processing Record 0 of Set 5 | malatya
Processing Record 1 of Set 5 | jining
Processing Record 2 of Set 5 | key largo
Processing Record 3 of Set 5 | providencia
Processing Record 4 of Set 5 | san patricio
Processing Record 5 of Set 5 | bandar-e anzali
City not fo

Processing Record 20 of Set 8 | dhidhdhoo
Processing Record 21 of Set 8 | airai
Processing Record 22 of Set 8 | kailua
Processing Record 23 of Set 8 | severo-kurilsk
Processing Record 24 of Set 8 | iskateley
Processing Record 25 of Set 8 | kon tum
Processing Record 26 of Set 8 | fort nelson
Processing Record 27 of Set 8 | solnechnyy
Processing Record 28 of Set 8 | yurginskoye
Processing Record 29 of Set 8 | saint-joseph
Processing Record 30 of Set 8 | bishop auckland
Processing Record 31 of Set 8 | bekessamson
Processing Record 32 of Set 8 | shashi
Processing Record 33 of Set 8 | mehamn
Processing Record 34 of Set 8 | novoagansk
Processing Record 35 of Set 8 | aranos
Processing Record 36 of Set 8 | moerai
Processing Record 37 of Set 8 | plettenberg bay
Processing Record 38 of Set 8 | bontang
Processing Record 39 of Set 8 | loukhi
Processing Record 40 of Set 8 | dakar
Processing Record 41 of Set 8 | tabiauea
City not found. Skipping...
Processing Record 42 of Set 8 | hopelchen
Processin

Processing Record 0 of Set 12 | biak
Processing Record 1 of Set 12 | leningradskiy
Processing Record 2 of Set 12 | tamale
Processing Record 3 of Set 12 | darnah
Processing Record 4 of Set 12 | north bend
Processing Record 5 of Set 12 | kupang
Processing Record 6 of Set 12 | flin flon
Processing Record 7 of Set 12 | tual
Processing Record 8 of Set 12 | comodoro rivadavia
Processing Record 9 of Set 12 | qarqin
Processing Record 10 of Set 12 | inhambane
Processing Record 11 of Set 12 | george
Processing Record 12 of Set 12 | luziania
Processing Record 13 of Set 12 | dalvik
Processing Record 14 of Set 12 | guerrero negro
Processing Record 15 of Set 12 | oktyabrskiy
Processing Record 16 of Set 12 | maningrida
Processing Record 17 of Set 12 | chumikan
Processing Record 18 of Set 12 | neyshabur
Processing Record 19 of Set 12 | beian
Processing Record 20 of Set 12 | lukovetskiy
Processing Record 21 of Set 12 | varena
Processing Record 22 of Set 12 | sarahan
Processing Record 23 of Set 12 | haa

In [8]:
#create new dataframe using new columns
weather_dict = {
    "City": city_column,
    "Cloudiness": cloudiness,
    "Country": country,
    "Date": date,
    "Humidity": humidity,
    "Lat": lat,
    "Lng": lng,
    "Max Temp": max_temp,
    "Wind Speed": wind_speed
}

In [9]:
weather_df = pd.DataFrame(weather_dict)
weather_df.head()

ValueError: arrays must all be same length

In [ ]:
#weather_df.to_csv(output_data_file)

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Max Temp"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Max Temperature (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.xlim(-60,80)
plt.ylim(-30,100)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig1.png")
plt.show()

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Humidity"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Humidity (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.ylim(15,105)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig2.png")
plt.show()

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Cloudiness"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Cloudiness (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig3.png")
plt.show()

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Wind Speed"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Wind Speed (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.ylim(-2,34)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig4.png")
plt.show()